In [25]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from fanalysis.pca import PCA
from sklearn.cluster import KMeans
from sklearn import preprocessing, metrics, model_selection, pipeline, compose
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer, InterclusterDistance
import matplotlib.pyplot as plt 
import missingno as msno
import scipy.cluster.hierarchy as shc
import seaborn as sns
import scipy.stats as stats
import time
import math
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [26]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

def read_clean_csv (df) : 
    path = Path(os.getcwd()) 
    p_parent = path.parent
    data_path = str(p_parent) + r"\data" + "\\" + str(df) + ".csv"
    df = pd.read_csv(str(data_path)).set_index("customer_unique_id")    
    clean_dataset(df)
    return df

def save_as(df, file_name) : 
    path = Path(os.getcwd()) 
    p_parent = path.parent
    data_path = str(p_parent) + r"\data" + "\\" + file_name + ".csv"
    df.to_csv(data_path, index = False)

In [27]:
path = Path(os.getcwd()) 
p_parent = path.parent
css6 = str(p_parent) + '\data\css6.csv'

css6 = pd.read_csv(css6)\
.set_index("customer_unique_id")

clean_dataset(css6)

css5.drop(columns=['not_defined'], inplace = True)

NameError: name 'css6' is not defined

In [ ]:
# normalisation MinMax
#scaler = MinMaxScaler()
scaler = StandardScaler()
segmentation_std = scaler.fit_transform(css6)

In [ ]:
wcss = []
for i in range(2,12):
    kmeans_pca = KMeans(n_clusters = i, init = 'k-means++', random_state=42)
    kmeans_pca.fit(segmentation_std)
    wcss.append(kmeans_pca.inertia_)
    
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(range(2,12),wcss, marker = 'o', linestyle ='--')
ax.set_xlabel("Nb. clusters")
ax.set_ylabel("WCSS")
plt.title("Kmean with PCA clustering")
plt.show()

In [ ]:
# Perform K-Means clustering
n_clusters = 4
kmean_pca = KMeans(n_clusters = n_clusters, init = 'k-means++', random_state=42)
kmean_pca.fit(segmentation_std)

In [ ]:
#col_name = []
#for i in range(1,n_components+1):
#    name = "Componant " + str(i)
#    col_name.append(name)

df_segm_pca_kmeans = pd.concat([css5.reset_index(drop = True), pd.DataFrame(segmentation_std)], axis = 1)

df_segm_pca_kmeans['Segment K-means PCA'] = kmean_pca.labels_

In [ ]:
df_segm_pca_kmeans.describe()

In [ ]:
df_segm_pca_kmeans['Segment'] = df_segm_pca_kmeans['Segment K-means PCA'].map({0:'first',
                                                                             1:'second',
                                                                             2:'third',
                                                                             3: 'fourth',
                                                                             4: 'fifth'
                                                                             })

x_axis = df_segm_pca_kmeans['Componant 2']
y_axis = df_segm_pca_kmeans['Componant 1']
z_axis = df_segm_pca_kmeans['Componant 3']


plt.style.use('default')
plt.figure(figsize =(10,8))
sns.scatterplot(x_axis, y_axis, hue = df_segm_pca_kmeans['Segment'], palette = ['g', 'r', 'c','m', 'y'])
plt.title('Cluster by PCA components')
plt.show()

plt.figure(figsize =(10,8))
sns.scatterplot(y_axis, z_axis, hue = df_segm_pca_kmeans['Segment'], palette = ['g', 'r', 'c','m', 'y'])
plt.title('Cluster by PCA components')
plt.show()

plt.figure(figsize =(10,8))
sns.scatterplot(x_axis, z_axis, hue = df_segm_pca_kmeans['Segment'], palette = ['g', 'r', 'c','m', 'y'])
plt.title('Cluster by PCA components')
plt.show()